In [3]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, precision_score,recall_score, f1_score
#from sklearn.metrics import roc_curve, roc_auc_score
#from sklearn.metrics import auc
import matplotlib.pyplot as plt

In [7]:
product_train= pd.read_csv("product_train.csv")
#p_test=pd.read_csv("product_test.csv")

In [8]:
def change(row):
    if row['went_on_backorder']=='No':
        val=0
    else:
        val=1
    return val
product_train['went_on_backorder']=product_train.apply(change,axis=1)


In [9]:
cat=['deck_risk','oe_constraint','ppap_risk','stop_auto_buy','rev_stop','potential_issue']
for i in cat:

    def change(row):
        if row['went_on_backorder']=='No':
            val=0
        else:
            val=1
        return val
    product_train[i]=product_train.apply(change,axis=1)

In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Example code for using SMOTE
X = product_train.drop(['went_on_backorder'], axis=1)  # Features
y = product_train['went_on_backorder']  # Target

# Create SMOTE object
smote = SMOTE(sampling_strategy='minority')

# Apply SMOTE to balance the dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=10)

In [11]:
product_test=pd.read_csv("product_test.csv")

In [12]:
import xgboost as xgb

In [13]:
param_xgb = {'n_estimators': [10,20,50,100,200,500],
             'learning_rate': [0.001,0.01,0.05, 0.1,0.5,1],
             'max_depth': [2,3,4,5,10,20,30,None],
             'min_child_weight': [5,25,50,100,500,1000],
             'gamma': [0,0.01,0.1,0.5,1,2,5],
             'subsample': [0.6,0.8,1.0],
             'colsample_bytree': [0.6,0.8,1.0],
             'objective': ['binary:logistic'],
             'nthread': [-1],
             'scale_pos_weight': [1,3,5,7,9]
             }

from sklearn.model_selection import RandomizedSearchCV

xgb_fit = RandomizedSearchCV(xgb.XGBClassifier(), cv = 5, param_distributions = param_xgb,
                                   scoring = 'f1', n_iter = 5, n_jobs = -1)
xgb_fit.fit(X_train, y_train)
xgb_fit.best_params_

{'subsample': 0.8,
 'scale_pos_weight': 7,
 'objective': 'binary:logistic',
 'nthread': -1,
 'n_estimators': 100,
 'min_child_weight': 5,
 'max_depth': 10,
 'learning_rate': 0.1,
 'gamma': 0,
 'colsample_bytree': 1.0}

In [14]:
xgb_fit.best_score_

0.9732865716294908

In [15]:
xgb_final_model = xgb.XGBClassifier(n_estimators = 100,learning_rate = 0.1,
                                    max_depth = 10,gamma = 0,subsample = 0.8,
                                    objective = 'binary:logistic',
                                    scale_pos_weight = 7,
                                    colsample_bytree = 1.0,
                                    min_child_weight = 5,
                                    nthread = -1)
xgb_final_model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, nthread=-1,
              num_parallel_tree=None, ...)

In [16]:
y_train_pred = xgb_final_model.predict(X_train)
print(" Below is result on training data with pruning : ")
print(accuracy_score(y_train,y_train_pred))
print(confusion_matrix(y_train,y_train_pred))
print(classification_report(y_train,y_train_pred))


# Test data
y_pred = xgb_final_model.predict(X_test)
print(" Below is result on test data with pruning : ")
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

 Below is result on training data with pruning : 
0.9763394878502792
[[189723   9335]
 [    71 198411]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.98    199058
           1       0.96      1.00      0.98    198482

    accuracy                           0.98    397540
   macro avg       0.98      0.98      0.98    397540
weighted avg       0.98      0.98      0.98    397540

 Below is result on test data with pruning : 
0.9728533193809993
[[46759  2646]
 [   52 49929]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     49405
           1       0.95      1.00      0.97     49981

    accuracy                           0.97     99386
   macro avg       0.97      0.97      0.97     99386
weighted avg       0.97      0.97      0.97     99386



In [17]:
from sklearn.model_selection import GridSearchCV

In [19]:
grid_search = GridSearchCV(xgb.XGBClassifier(), cv = 5, param_grid = param_xgb,
                        scoring = 'f1', n_jobs = -1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
xgb_final_model = xgb.XGBClassifier(n_estimators = 200,learning_rate = 0.5,
                                    max_depth = 4,gamma = 0.5,subsample = 0.6,
                                    objective = 'binary:logistic',
                                    scale_pos_weight = 9,
                                    colsample_bytree = 1.0,
                                    min_child_weight = 5,
                                    nthread = -1)
xgb_final_model.fit(X_train,y_train)

In [ ]:
uy_train_pred = xgb_final_model.predict(X_train)
print(" Below is result on training data with pruning : ")
print(accuracy_score(y_train,y_train_pred))
print(confusion_matrix(y_train,y_train_pred))
print(classification_report(y_train,y_train_pred))


# Test data
y_pred = xgb_final_model.predict(X_test)
print(" Below is result on test data with pruning : ")
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))